<a href="https://colab.research.google.com/github/balakg/ipy-vision/blob/main/notebooks/dsp/aliasing_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aliasing in 2D

When we downsample an image without proper filtering, we violate the Nyquist-Shannon sampling theorem, leading to aliasing. This demo showcases aliasing on sample images along with Fourier spectra visualizations.

### The Theory
1.  **Sampling as Multiplication**: In the spatial domain, downsampling is equivalent to multiplying the image by a 2D "comb" or "delta train" function.
2.  **Spectral Convolution**: In the frequency domain, this multiplication results in the convolution of the image's spectrum with another delta train. Effectively, the spectrum is **replicated** periodically.
3.  **Aliasing**: If the original spectrum contains frequencies higher than half the sampling rate (the Nyquist frequency), these replicas overlap. This causes high-frequency information to "fold back" and appear as low-frequency artifacts (like Moiré patterns).

### The Nyquist Limit
To prevent aliasing, we must ensure the signal is band-limited. For a downsampling factor of $S$, the maximum frequency $f_{max}$ must satisfy:

$$f_{max} < \frac{1}{2S}$$,

where $S$ is the signal's sampling rate.

### How to use this demo:
* **Downsample**: Increase the step size to see the spectral replicas move closer together.
* **Filter**: Select **Gaussian** to apply a low-pass filter. This suppresses the frequencies that would otherwise cause overlap.
* **Observe**: Notice how the downsampled spectrum becomes less "crowded" as you increase $\sigma$, but the image becomes blurrier.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, color
from scipy.fft import fft2, fftshift
from scipy.ndimage import gaussian_filter
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import output
output.no_vertical_scroll()


# 1. Define available images from skimage.data
image_options = {
    'Astronaut': data.astronaut,
    'Rocket': data.rocket,
    'Camera': data.camera,
    'Coins': data.coins,
    'Brick': data.brick,
    'Grass': data.grass,
    'Checkerboard': data.checkerboard
}

out = widgets.Output()

def get_spectrum(image, target_shape):
    """Computes magnitude spectrum padded to target_shape."""
    f_coeff = fftshift(fft2(image))
    magnitude_spectrum = np.log(1 + np.abs(f_coeff))

    res = np.zeros(target_shape)
    h, w = magnitude_spectrum.shape
    th, tw = target_shape

    h_slice, w_slice = min(h, th), min(w, tw)
    r_start, c_start = (th - h_slice) // 2, (tw - w_slice) // 2

    res[r_start:r_start+h_slice, c_start:c_start+w_slice] = \
        magnitude_spectrum[:h_slice, :w_slice]
    return res

def update_demo(*args):
    # 2. Get image based on dropdown selection
    selected_img_func = image_options[image_dropdown.value]
    base_img = selected_img_func()

    # Preprocess to grayscale
    if base_img.ndim == 3:
        base_img = color.rgb2gray(base_img)

    step = step_slider.value
    blur_type = filter_dropdown.value
    sigma = sigma_slider.value

    # Apply Anti-Aliasing Filter
    if blur_type == 'Gaussian':
        processed_img = gaussian_filter(base_img, sigma=sigma, mode='wrap')
    else:
        processed_img = base_img.copy()

    # 1. Downsampled: Standard decimation
    downsampled = processed_img[::step, ::step]

    # 2. Zero-inserted: Expand back to original size
    zero_inserted = np.zeros_like(base_img)
    zero_inserted[::step, ::step] = downsampled

    # Compute spectra
    orig_sp = get_spectrum(processed_img, base_img.shape)
    zero_sp = get_spectrum(zero_inserted, base_img.shape)
    down_sp = get_spectrum(downsampled, base_img.shape)

    with out:
        clear_output(wait=True)
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))

        img_title = fr"Input Image (Gaussian $\sigma$ = {sigma:.2f})" if blur_type == 'Gaussian' else "Input Image"

        axes[0, 0].imshow(processed_img, cmap='gray')
        axes[0, 0].set_title(img_title, fontsize=14, fontweight='bold')

        axes[0, 1].imshow(zero_inserted, cmap='gray')
        axes[0, 1].set_title(f"Image x Delta Train (every {step} px)", fontsize=14, fontweight='bold')

        axes[0, 2].imshow(downsampled, cmap='gray')
        axes[0, 2].set_title(f"Downsampled {step}x (Upsampled for Vis)", fontsize=14, fontweight='bold')

        mn, mx = orig_sp.min(), orig_sp.max()

        # Row 2: Spectra
        axes[1, 0].imshow(orig_sp, vmin=mn, vmax=mx, cmap='Blues')
        axes[1, 0].set_title("Input Image Spectrum", fontsize=14, fontweight='bold')

        axes[1, 1].imshow(zero_sp, vmin=mn, vmax=mx, cmap='Blues')
        axes[1, 1].set_title("Image x Delta Train Spectrum", fontsize=14, fontweight='bold')

        axes[1, 2].imshow(down_sp, vmin=mn, vmax=mx, cmap='Blues')
        axes[1, 2].set_title("Downsampled Spectrum", fontsize=14, fontweight='bold')

        for ax in axes.flatten(): ax.axis('off')
        plt.tight_layout(h_pad=2.0)
        plt.show()


# Stylized Title
title_widget = widgets.HTML("""
<h2 style="color: #ffffff; margin-top: 0px; margin-bottom: 5px; font-family: sans-serif;">
    2D Aliasing & Sampling Demo
</h2>
<p style="color: #7f8c8d; margin-top: 0px; font-family: sans-serif; margin-bottom: 15px;">
    Adjust sliders to visualize how undersampling folds the frequency spectrum.
</p>
""")


# Widget setup
image_dropdown = widgets.Dropdown(
    options=list(image_options.keys()),
    value='Astronaut',
    description='Image:'
)
step_slider = widgets.IntSlider(value=2, min=1, max=10, description='Downsample:', continuous_update=False)
filter_dropdown = widgets.Dropdown(
    options=['None', 'Gaussian'],
    value='None',
    description='Filter:'
)
sigma_slider = widgets.FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Blur Sigma:', continuous_update=False)

# Link all widgets
widgets.interactive_output(update_demo, {
    'image': image_dropdown,
    'step': step_slider,
    'filter': filter_dropdown,
    'sigma': sigma_slider
})

# Setting up observers for the manual call
for w in [image_dropdown, step_slider, filter_dropdown, sigma_slider]:
    w.observe(update_demo, names='value')

# Combine into a clean dashboard
controls = widgets.VBox([
    title_widget,
    image_dropdown,
    step_slider,
    filter_dropdown,
    sigma_slider
], layout=widgets.Layout(
    padding='10px',
    border='1px solid #ddd',
    border_radius='10px',
    margin='0 0 25px 0',
    background_color='#fcfcfc',
    max_width='fit-content'
))

# Display Everything
out.layout.min_height = '50px' # Prevent scrolling in Colab
display(controls, out)
update_demo()